In [ ]:
!pip install watson-machine-learning-client --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 538 kB 12.6 MB/s 
     |████████████████████████████████| 55 kB 3.6 MB/s 
     |████████████████████████████████| 132 kB 69.8 MB/s 
     |████████████████████████████████| 9.9 MB 61.5 MB/s 
     |████████████████████████████████| 79 kB 7.5 MB/s 
     |████████████████████████████████| 140 kB 48.3 MB/s 
     |████████████████████████████████| 956 kB 56.1 MB/s 
     |████████████████████████████████| 135 kB 55.4 MB/s 
     |████████████████████████████████| 62 kB 1.5 MB/s 
  Created wheel for ibm-cos-sdk: filename=ibm_cos_sdk-2.12.0-py3-none-any.whl size=73931 sha256=684c697dc51a164644a4972100f3649b2a609fa6e775c62421f514a60a797bf7
  Stored in directory: /root/.cache/pip/wheels/ec/94/29/2b57327cf00664b6614304f7958abd29d77ea0e5bbece2ea57
  Created wheel for ibm-cos-sdk-core: filename=ibm_cos_sdk_core-2.12.0-py3-none-any.whl size=562962 sha256=82ba237ffc

In [1]:
!unzip '/content/sample_data/Augment_data.zip'

Archive:  /content/sample_data/Augment_data.zip
   creating: content/Augment_data/
   creating: content/Augment_data/GIB_AUG/
  inflating: content/Augment_data/GIB_AUG/aug_download (2)_0_6629.jpg  
  inflating: content/Augment_data/GIB_AUG/aug_download (6)_0_3500.jpg  
  inflating: content/Augment_data/GIB_AUG/aug_images (1)_0_695.jpg  
  inflating: content/Augment_data/GIB_AUG/aug_images (7)_0_6849.jpg  
  inflating: content/Augment_data/GIB_AUG/aug_download (5)_0_3065.jpg  
  inflating: content/Augment_data/GIB_AUG/aug_download (9)_0_5970.jpg  
  inflating: content/Augment_data/GIB_AUG/aug_download (7)_0_9545.jpg  
  inflating: content/Augment_data/GIB_AUG/aug_download (10)_0_9609.jpg  
  inflating: content/Augment_data/GIB_AUG/aug_download (6)_0_2250.jpg  
  inflating: content/Augment_data/GIB_AUG/aug_download (10)_0_2022.jpg  
  inflating: content/Augment_data/GIB_AUG/aug_images (7)_0_7397.jpg  
  inflating: content/Augment_data/GIB_AUG/aug_download (8)_0_3871.jpg  
  inflating: co

## **Importing Libraries**

In [2]:
import os, gc, glob, random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import keras
from PIL import Image
from keras.layers import Dense,Flatten
from keras.models import Model
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split

In [3]:
base_model = InceptionV3(input_shape=(224,224,3),include_top=False)


87910968/87910968 [==============================] - 0s 0us/step


In [5]:
for layer in base_model.layers:
  layer.trainable = False

## **Flatening and adding dense layer**

In [6]:
X = Flatten()(base_model.output)

#Using 'sigmoid function' as activation funciton
X = Dense(units = 6, activation = 'sigmoid')(X)

## **Compiling the model**

In [7]:
model = Model(base_model.input,X)

# Using adam as the optimizer
# Accuracy is set as the metrics to judge the performance of the model
model.compile(optimizer = 'adam',loss = keras.losses.binary_crossentropy,metrics=['accuracy'])

# Summarizing the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 111, 111, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                             

## **loading the train and test dataset**

In [14]:
#initializing the datagenerator
datagen = tf.keras.preprocessing.image.ImageDataGenerator()

#loading the train dataset from the directory
traindata= datagen.flow_from_directory(directory="/content/Train_set/Train_data",target_size=(224,224),batch_size=100)

#loading the test dataset from the directory
testdata= datagen.flow_from_directory(directory="/content/Train_set/Test_data",target_size=(224,224),batch_size=100)


Found 3378 images belonging to 6 classes.
Found 1318 images belonging to 6 classes.


In [15]:
#listing the found classes from train dataset
traindata.class_indices

{'Corpse_AUG': 0,
 'GIB_AUG': 1,
 'LS_Orchid_AUG': 2,
 'LS_Pangolin_AUG': 3,
 'SPS_AUG': 4,
 'SW_Deer_AUG': 5}

## **Model Checkpoint and Early Stopping**

In [16]:
# Creating a model checkpoint which monitors the accuracy of the model and saves the checkpoint
mc = ModelCheckpoint(filepath = "./model.h5",
                     monitor = 'accuracy',
                     verbose = 1,
                     save_best_only = True)

# Creating a earlystopping object which stop training once the model performance stops improving on a hold out validation dataset
es = EarlyStopping(monitor = "accuracy",
                   min_delta = 0.01,
                   verbose = 1)

call_back = [mc,es]

## **Model fitting**

In [19]:
modelHistory = model.fit(traindata, steps_per_epoch=60,epochs = 30,callbacks=call_back,validation_data=testdata)


Epoch 1/30
34/60 [================>.............] - ETA: 4:07 - loss: 1.1534 - accuracy: 0.9008


Epoch 1: accuracy improved from 0.76702 to 0.90083, saving model to ./model.h5
60/60 [==============================] - 449s 7s/step - loss: 1.1534 - accuracy: 0.9008 - val_loss: 0.8294 - val_accuracy: 0.8892


## **Exporting the model**

In [20]:
model_json = model.to_json()
with open("DigitalNaturalist.json", "w") as json_file:
    json_file.write(model_json)

# Exporting the model weights

model.save_weights("DigitalNaturalist")
print("Saved model to disk")


Saved model to disk


## **Testing the model**

In [25]:
#Testing the model 
predictions = ["Corpse Flower", 
               "Great Indian Bustard", 
               "Lady's slipper orchid", 
               "Pangolin", 
               "Spoon Billed Sandpiper", 
               "Seneca White Deer"
              ]


path = '/content/Train_set/Test_data/LS_Orchid_AUG/aug_download (1)_0_1502.jpg'
ime = tf.keras.utils.load_img(path,target_size=(224,224))

i = tf.keras.preprocessing.image.img_to_array(ime)
i = preprocess_input(i)
input = np.array([i])
pred = model.predict(input)
predictions[np.argmax(pred)]


1/1 [==============================] - 0s 149ms/step


"Lady's slipper orchid"